#augment training photos

In [ ]:
#a project folder must be created with a special folder structure
#as documented

import os #operating system
import shutil #copy files
from PIL import Image #image processing (size, rotate etc)
import fnmatch

cwd = os.getcwd()

print('current working directory: ',cwd)
answer = input('Should the current working directory change? (y/n): ')

if answer == 'y':
    cwd = input('Please enter the path which should be used: ')
    os.chdir(cwd)
    print('current working directory: ', cwd)
else:
    print('working directory not changed')

In [ ]:
#take a look in every folder
#test if there are more than 100 pictures in very folder for training
    #if there are more than 100 photos in a folder then
        #change every picture in this folder to that size which is needed in the network
    #if there are less than 100 photos in a folder
        #flip every photo left-right and create a new photo
        #rotate every photo 5 degree clockwise and counter clockwise (-> creating new photos)
#test
print(cwd)
#change in the training folder
cwd = os.path.join(cwd, 'training')
print(cwd)
os.chdir(cwd)

dir_list = os.listdir(cwd)
#test
print(dir_list)

#define the new size of the input pictures for the training
network_img_size = int(input('which size the images should have as the network input?: '))

for i in dir_list:
    #how many files in each class for training available
    number_files = len(os.listdir(cwd + '\\' + i))
    #test
    print(number_files)
    
    #biggest height/ width
    #dst_directory = cwd + '\originalphotosnetwork'
    files = os.listdir(cwd + '\\' + i)
    max_width = 0
    max_height = 0
    for f in files:
        img = Image.open(cwd + '\\' + i + '\\' + f, 'r')
        width, height = img.size
        if width > max_width:
            max_width = width
        if height > max_height:
            max_height = height
        if height == 0 or width == 0:
            print('width or height = 0 -> no processing possible')
        if max_height < 224 or max_width < 224:
            print('height or width of an image are less than 224 pixels -> no processing possible')
    #test    
    #print(max_width, ', ', max_height)

    max_pixel = max([max_width, max_height])
    #test
    #print(max_pixel)

    #converte all photos to squares with the max out of height and width for
    #pixels

    new_size = max_pixel, max_pixel
    for f in files:
        old_img = Image.open(cwd + '\\' + i + '\\' + f, 'r')
        old_size = old_img.size
        new_img = Image.new('RGB', new_size)
        #the tuple in paste defines the upper left corner. to place the image in
        #the center use the calculation
        position_width = int((new_size[0] - old_size[0]) / 2)
        position_height = int((new_size[1] - old_size[1]) / 2)
        new_img.paste(old_img, (position_width, position_height))
        #test
        #new_img.show()
        new_img.save(cwd + '\\' + i + '\\' + f)

    #now give the images the size for the network
    for f in files:
        old_img = Image.open(cwd + '\\' + i + '\\' + f, 'r')
        #ANTIALIAS should be a great downsizing filter
        new_img = old_img.resize((network_img_size, network_img_size), Image.ANTIALIAS)
        #test
        #new_img.show()
        new_img.save(cwd + '\\' + i + '\\' + f)
        
    #flipping the images in every class to double the training images
    for f in files:
        old_img = Image.open(cwd + '\\' + i + '\\' + f, 'r')
        new_img = old_img.transpose(Image.FLIP_LEFT_RIGHT)
        new_img.save(cwd + '\\' + i + '\\' + '2' + f)
        
    #rotate the trainings pictures one degree clock wise and counter clock
    #wise to create new learning images
    for f in files:
        for degree in range(1, 5):
            old_img = Image.open(cwd + '\\' + i + '\\' + f, 'r')
            new_img = old_img.rotate(degree)
            new_img.save(cwd + '\\' + i + '\\' + str(degree) + f)
            new_img = old_img.rotate(360 - degree)
            new_img.save(cwd + '\\' + i + '\\' + str(360 - degree) + f)